# Add and norm

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_add_and_norm1.png" alt="Add and norm" style="width:425px;height:626px;">
</div>

Una vez hemos pasado el bloque de atención del dencoder, vemos que la salida pasa por una capa llamada `Add & Norm`, a la que además le entra la matriz que entra al módulo de atención. A esto se le llama conexión residual

## Conexiones residuales

la idea de las conexiones residuales fue introducida por primera vez en la arquitectura `ResNet` (Redes Neuronales Residuales), propuesta por Kaiming He y sus colegas de Microsoft Research en 2015. La idea clave en ResNet es introducir `conexiones de salto` (también conocidas como conexiones residuales o atajos) que permiten a los gradientes fluir directamente a través de las capas.

Estas conexiones de salto permiten a la red aprender de una manera más sencilla, lo que a su vez ayuda a combatir el problema del desvanecimiento del gradiente en redes muy profundas. Esta característica ha permitido el entrenamiento de redes neuronales convolucionales de hasta 152 capas, mientras que anteriormente las redes estaban limitadas a unas pocas decenas de capas.

Las conexiones residuales aportan múltiples beneficios, tanto durante el entrenamiento como durante la inferencia.

 * Durante el entrenamiento:

   * Alivian el problema del desvanecimiento del gradiente: Las conexiones residuales permiten el paso de los gradientes directamente a través de las capas, lo que ayuda a mantenerlos lo suficientemente grandes para que el modelo pueda seguir aprendiendo, incluso en las capas más profundas.

   * Permiten el entrenamiento de redes más profundas: Al ayudar a mitigar el problema del desvanecimiento del gradiente, las conexiones residuales también facilitan el entrenamiento de redes más profundas, lo cual puede llevar a mejor rendimiento.

 * Durante la inferencia:

   * Permiten la explotación de información a diferentes niveles de abstracción: Dado que las conexiones residuales permiten que la salida de cada capa sea la suma de la entrada y la transformación aprendida, la información de diferentes niveles de abstracción puede ser explotada. Esto puede ser beneficioso en muchas tareas, especialmente en las que la información de bajo y alto nivel puede ser útil.

   * Mejoran la robustez del modelo: Dado que las conexiones residuales permiten que las capas aprendan transformaciones residuales (es decir, diferencias o "correcciones" a la identidad), los modelos con conexiones residuales pueden ser más robustos a perturbaciones en los datos de entrada.

   * Permiten la recuperación de información perdida: Si alguna información se pierde durante la transformación en alguna capa, las conexiones residuales pueden permitir que esta información sea recuperada en las capas posteriores.

## Add and Norm

Lo que se hace es sumar la matriz que entra al módulo de atención y la que sale del módulo de atención. Además se realiza un normalización, restando la media y dividiendo entre la desviación estandar para evitar tener matrices con valores grandes

## Implementación

Vamos a implementar una clase que hará el módulo de `Add & Norm`

In [7]:
import torch
import torch.nn as nn

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

Vamos a coger el embbeding preentrenado de BERT, la implementación de la capa de atención que hemos hecho antes y vamos a ver qué obtenemos a la salida de nuestra clase

In [8]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V):
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Obtenemos el resultado del input embedding más el positional encoding

In [9]:
sentence1 = "I gave the dog a bone because it was hungry"
tokens1, input_ids1, token_embeddings1, positional_encodings1, embeddings_with_positional_encoding1 = extract_embeddings(sentence1)

Instanciamos un objeto de la clase `Multi-Head Attention` y obtenemos su salida

In [10]:
dim_embedding = embeddings_with_positional_encoding1.shape[-1]
heads = 8
multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)
attention = multi_head_attention(embeddings_with_positional_encoding1, embeddings_with_positional_encoding1, embeddings_with_positional_encoding1)

Ahora instanciamos un objeto de la clase `Add & Norm`

In [11]:
add_and_norm = AddAndNorm(dim_embedding=dim_embedding)

Le metemos la matriz de antes de la capa de atención y la de la salida de la capa de atención

In [12]:
add_and_norm_output = add_and_norm(embeddings_with_positional_encoding1, attention)
add_and_norm_output.shape

torch.Size([1, 12, 768])

Seguimos obteniendo una matriz de 1x12x768, parece que todo bien